## Introduction

This notebook presents the development of a  ml model designed to predict whether a mud pump is likely to stop operating within a chosen time horizon.
Chosen horizon was 7 days, I analyzed and model data from Mud Pump A and Mud Pump B, exploring their operational characteristics, engineering relevant features, training a classifier, and evaluating its performance.

#### The workflow covered in this notebook includes:

<li>Data cleaning and preparation

<li>Feature engineering and leakage prevention

<li>Construction of a unified modelling dataset

<li>Training a classification model

<li>Performance evaluation through metrics and a confusion matrix
<br>
A more detailed methodology, including assumptions,other methods and  decisions is documented in the accompanying report:

🔗 Detailed Report: See the full documentation in
./reports/<your-report-file.pdf>

In [78]:
#importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from feature_engine.selection import DropCorrelatedFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report


In [79]:
#reading the dataset
a = pd.read_excel('PredictiveMaintenanceTestData.xlsx', sheet_name='Mud Pump A Data')
a_2 = pd.read_excel('PredictiveMaintenanceTestData.xlsx', sheet_name='Mud Pump A Data2')
b = pd.read_excel('PredictiveMaintenanceTestData.xlsx', sheet_name='Mud Pump B Data')
b_2 = pd.read_excel('PredictiveMaintenanceTestData.xlsx', sheet_name='Mud Pump B Data2')


In [80]:
a.head()

,Datetime,Mud Pump A,React 55 Mud Flow to Mud Pump A,MUD PUMP A SUCTION PRESS,MUD PUMP A SPEED,MUD PUMP A SPEED.1,PUM-34A- Current,Mud pump A speed,MUD PUMP A CYLINDER 2,MUD PUMP A CYLINDER 3,Mud Pump A Cylinder #1 Temperature,Mud Pump A Cylinder #2 Temperature,Mud Pump A Cylinder #3 Temperature,Mud Pump A Cylinder #4 Temperature
0,2024-09-03 00:00:00,RUN,105.130142,387.835968,42.003342,99.395256,262.888275,41.446602,6773.156250,6795.823242,62.210194,52.913517,59.367279,57.872066
1,2024-09-03 01:00:00,RUN,145.442825,434.914001,42.046818,99.322189,268.082886,41.446602,7006.632324,7035.887695,61.636726,51.343925,59.750381,58.145172
2,2024-09-03 02:00:00,RUN,140.586426,424.093658,41.957718,99.467384,265.449982,41.446602,6818.970215,6843.666016,61.427448,52.121159,60.373962,58.975826
3,2024-09-03 03:00:00,RUN,148.379059,423.871460,42.01181,99.364395,270.196472,41.446602,6922.452148,6951.882324,61.100765,49.370907,59.966591,58.109085
4,2024-09-03 04:00:00,RUN,140.214325,425.911072,41.952599,99.500214,269.50528,41.446602,6925.411133,6958.680664,61.523418,50.280827,61.684792,58.280701


In [81]:
b_2.head()

,Date,B_LAST_NORTH_OIL_SIDE_SM,B_LAST_SOUTH_OIL_SIDE_SM,B_NORTH_OIL_SIDE.DAYS_LEFT,B_NORTH_OIL_SIDE.EXPECTED_LIFE,B_NORTH_OIL_SIDE.RESID_LIFE_%,B_NORTH_OIL_SIDE.RLIFE_STATUS,B_NORTH_OIL_SIDE.RUNDAYS,B_SOUTH_OIL_SIDE.DAYS_LEFT,B_SOUTH_OIL_SIDE.EXPECTED_LIFE,B_SOUTH_OIL_SIDE.RESID_LIFE_%,B_SOUTH_OIL_SIDE.RLIFE_STATUS,B_SOUTH_OIL_SIDE.RUNDAYS
0,2024-09-03,2023-07-12,2023-07-12,90.477127,365,24.799681,ON_LINE,274.522888,90.477127,365,24.799681,ON_LINE,274.522888
1,2024-09-04,2023-07-12,2023-07-12,89.477127,365,24.525707,ON_LINE,275.522888,89.477127,365,24.525707,ON_LINE,275.522888
2,2024-09-05,2023-07-12,2023-07-12,88.477127,365,24.251732,ON_LINE,276.522888,88.477127,365,24.251732,ON_LINE,276.522888
3,2024-09-06,2023-07-12,2023-07-12,87.477127,365,23.977762,ON_LINE,277.522888,87.477127,365,23.977762,ON_LINE,277.522888
4,2024-09-07,2023-07-12,2023-07-12,86.477127,365,23.703787,ON_LINE,278.522888,86.477127,365,23.703787,ON_LINE,278.522888


In [82]:

def fix_datetime(df, datetime_col):
    """
    convert a datetime column to pandas datetime format
    and create a daily 'date' column for merging.
    
    parameters:
        df (pd.DataFrame): input dataframe
        datetime_col (str): name of the datetime column in the dataframe
        
    returns:
        pd.DataFrame: dataframe with datetime fixed and daily 'date' column added
    """
    # convert to pandas datetime
    df[datetime_col] = pd.to_datetime(df[datetime_col])
    
    # extract date only for daily merging
    df['date'] = df[datetime_col].dt.date
    
    return df


a = fix_datetime(a, 'Datetime')
a_2 = fix_datetime(a_2, 'Date')
b = fix_datetime(b, 'Datetime')
b_2 = fix_datetime(b_2, 'Date')


In [83]:
a.head()


,Datetime,Mud Pump A,React 55 Mud Flow to Mud Pump A,MUD PUMP A SUCTION PRESS,MUD PUMP A SPEED,MUD PUMP A SPEED.1,PUM-34A- Current,Mud pump A speed,MUD PUMP A CYLINDER 2,MUD PUMP A CYLINDER 3,Mud Pump A Cylinder #1 Temperature,Mud Pump A Cylinder #2 Temperature,Mud Pump A Cylinder #3 Temperature,Mud Pump A Cylinder #4 Temperature,date
0,2024-09-03 00:00:00,RUN,105.130142,387.835968,42.003342,99.395256,262.888275,41.446602,6773.156250,6795.823242,62.210194,52.913517,59.367279,57.872066,2024-09-03
1,2024-09-03 01:00:00,RUN,145.442825,434.914001,42.046818,99.322189,268.082886,41.446602,7006.632324,7035.887695,61.636726,51.343925,59.750381,58.145172,2024-09-03
2,2024-09-03 02:00:00,RUN,140.586426,424.093658,41.957718,99.467384,265.449982,41.446602,6818.970215,6843.666016,61.427448,52.121159,60.373962,58.975826,2024-09-03
3,2024-09-03 03:00:00,RUN,148.379059,423.871460,42.01181,99.364395,270.196472,41.446602,6922.452148,6951.882324,61.100765,49.370907,59.966591,58.109085,2024-09-03
4,2024-09-03 04:00:00,RUN,140.214325,425.911072,41.952599,99.500214,269.50528,41.446602,6925.411133,6958.680664,61.523418,50.280827,61.684792,58.280701,2024-09-03


In [84]:
b_2.head()

,Date,B_LAST_NORTH_OIL_SIDE_SM,B_LAST_SOUTH_OIL_SIDE_SM,B_NORTH_OIL_SIDE.DAYS_LEFT,B_NORTH_OIL_SIDE.EXPECTED_LIFE,B_NORTH_OIL_SIDE.RESID_LIFE_%,B_NORTH_OIL_SIDE.RLIFE_STATUS,B_NORTH_OIL_SIDE.RUNDAYS,B_SOUTH_OIL_SIDE.DAYS_LEFT,B_SOUTH_OIL_SIDE.EXPECTED_LIFE,B_SOUTH_OIL_SIDE.RESID_LIFE_%,B_SOUTH_OIL_SIDE.RLIFE_STATUS,B_SOUTH_OIL_SIDE.RUNDAYS,date
0,2024-09-03,2023-07-12,2023-07-12,90.477127,365,24.799681,ON_LINE,274.522888,90.477127,365,24.799681,ON_LINE,274.522888,2024-09-03
1,2024-09-04,2023-07-12,2023-07-12,89.477127,365,24.525707,ON_LINE,275.522888,89.477127,365,24.525707,ON_LINE,275.522888,2024-09-04
2,2024-09-05,2023-07-12,2023-07-12,88.477127,365,24.251732,ON_LINE,276.522888,88.477127,365,24.251732,ON_LINE,276.522888,2024-09-05
3,2024-09-06,2023-07-12,2023-07-12,87.477127,365,23.977762,ON_LINE,277.522888,87.477127,365,23.977762,ON_LINE,277.522888,2024-09-06
4,2024-09-07,2023-07-12,2023-07-12,86.477127,365,23.703787,ON_LINE,278.522888,86.477127,365,23.703787,ON_LINE,278.522888,2024-09-07


In [85]:

def prepare_daily(
    df: pd.DataFrame,
    datetime_col: str,
    keep_non_numeric: bool = True,
    non_numeric_method: str = "first",
    coerce_numeric: bool = True,
    date_name: str = "date"
) -> pd.DataFrame:
    """
    converts timestamp column to daily frequency and aggregate numeric columns
    by mean. Optionally keep non-numeric columns by taking the first value (or mode).
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe.
    datetime_col : str
        Name of the datetime column in the dataframe.
    keep_non_numeric : bool, default True
        Whether to keep non-numeric columns in the output (aggregated by 'first' or 'mode').
    non_numeric_method : {'first', 'mode'}, default 'first'
        How to aggregate non-numeric columns if keep_non_numeric is True.
    coerce_numeric : bool, default True
        If True, attempt to coerce possible-numeric object columns to numeric (errors -> NaN).
    date_name : str, default 'date'
        Name for the resulting date column (daily), will be dtype datetime64[ns] at midnight.
    
    Returns
    -------
    pd.DataFrame
        Daily-aggregated dataframe. Index is default integer, with a `date` column.
    """
    df = df.copy()
    
   
    df[datetime_col] = pd.to_datetime(df[datetime_col], errors="coerce")
    if df[datetime_col].isna().all():
        raise ValueError(f"All values in {datetime_col} could not be parsed to datetime.")
    
    # Create date (midnight timestamp)
    df[date_name] = df[datetime_col].dt.normalize()  
    
    if coerce_numeric:
        for col in df.columns:
            if df[col].dtype == "object":
                # Attempt to convert; non-convertible become NaN
                df[col] = pd.to_numeric(df[col], errors="coerce").combine_first(df[col])
                # combine_first keeps original strings where conversion failed
    
    numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
    
    if len(numeric_cols) == 0:
        # nothing numeric to aggregate — return first values per day (or raise)
        numeric_agg = pd.DataFrame(index=pd.Index([], name=date_name))
    else:
        numeric_agg = df.groupby(date_name)[numeric_cols].mean()
    
    # Handle non-numeric columns if requested
    if keep_non_numeric:
        non_numeric_cols = [c for c in df.columns if c not in numeric_cols + [datetime_col, date_name]]
        non_numeric_agg = {}
        for col in non_numeric_cols:
            if non_numeric_method == "first":
                non_numeric_agg[col] = df.groupby(date_name)[col].first()
            elif non_numeric_method == "mode":
                # mode may return multiple; take the first mode if exists, else NaN
                non_numeric_agg[col] = df.groupby(date_name)[col].agg(
                    lambda s: (s.mode().iloc[0] if not s.mode().empty else pd.NA)
                )
            else:
                raise ValueError("non_numeric_method must be 'first' or 'mode'")
        if non_numeric_agg:
            non_numeric_agg_df = pd.concat(non_numeric_agg.values(), axis=1)
            non_numeric_agg_df.columns = list(non_numeric_agg.keys())
            # numeric + non-numeric
            if isinstance(numeric_agg, pd.DataFrame) and not numeric_agg.empty:
                daily = pd.concat([numeric_agg, non_numeric_agg_df], axis=1)
            else:
                daily = non_numeric_agg_df
        else:
            daily = numeric_agg
    else:
        daily = numeric_agg
    
   
    if isinstance(daily, pd.DataFrame):
        daily = daily.reset_index()
    else:
        daily = daily.to_frame().reset_index()
        daily[date_name] = pd.to_datetime(daily[date_name])
    
    return daily


In [86]:
daily_a1 = prepare_daily(a, 'Datetime', keep_non_numeric=True, non_numeric_method='first')

daily_a2 = prepare_daily(a_2, 'Date', keep_non_numeric=True, non_numeric_method='first')
daily_b1 = prepare_daily(b, 'Datetime', keep_non_numeric=True, non_numeric_method='first')

daily_b2 = prepare_daily(b_2, 'Date', keep_non_numeric=True, non_numeric_method='first')

merged_a = daily_a1.merge(daily_a2, on='date', how='inner')
merged_b = daily_b1.merge(daily_b2, on='date', how='inner')

In [87]:

merged_a.head()

,date,React 55 Mud Flow to Mud Pump A,MUD PUMP A SUCTION PRESS,MUD PUMP A SPEED.1,MUD PUMP A CYLINDER 2,MUD PUMP A CYLINDER 3,Mud Pump A Cylinder #1 Temperature,Mud Pump A Cylinder #2 Temperature,Mud Pump A Cylinder #3 Temperature,Mud Pump A Cylinder #4 Temperature,...,A_NORTH.LAST_MINI_OIL_SIDE_SM,A_NORTH.LAST_OIL_SIDE_SM,A_NORTH_MINI_OIL_SIDE.RLIFE_STATUS,A_NORTH_OIL_SIDE.RLIFE_STATUS,A_SOUTH.LAST_MINI_OIL_SIDE_SM,A_SOUTH.LAST_OIL_SIDE_SM,A_SOUTH_MINI_OIL_SIDE.RLIFE_STATUS,A_SOUTH_OIL_SIDE.RLIFE_STATUS,A_LAST_MUD_SIDE_SM,A_MUD_SIDE.RLIFE_STATUS
0,2024-09-03,124.984502,397.001902,95.775800,6775.664246,6801.475850,62.122698,52.474178,61.062826,57.914323,...,2023-03-31,2023-01-18,ON_LINE,ON_LINE,2023-03-31,2023-01-18,ON_LINE,ON_LINE,2016-01-08,ON_LINE
1,2024-09-04,104.673728,355.022669,94.778817,6579.124349,6603.233887,62.024733,52.407360,60.802001,56.761674,...,2023-03-31,2023-01-18,ON_LINE,ON_LINE,2023-03-31,2023-01-18,ON_LINE,ON_LINE,2016-01-08,ON_LINE
2,2024-09-05,96.006373,358.586122,90.898222,6390.380920,6415.289510,60.754139,50.022266,59.948708,54.314469,...,2023-03-31,2023-01-18,ON_LINE,ON_LINE,2023-03-31,2023-01-18,ON_LINE,ON_LINE,2016-01-08,ON_LINE
3,2024-09-06,88.442687,372.479466,87.573452,6779.555349,6803.085571,61.347060,51.532399,60.425867,55.867826,...,2023-03-31,2023-01-18,ON_LINE,ON_LINE,2023-03-31,2023-01-18,ON_LINE,ON_LINE,2016-01-08,ON_LINE
4,2024-09-07,99.902062,373.857407,93.896962,6934.170593,6958.844666,63.144880,54.056685,62.013078,58.409987,...,2023-03-31,2023-01-18,ON_LINE,ON_LINE,2023-03-31,2023-01-18,ON_LINE,ON_LINE,2016-01-08,ON_LINE


In [88]:
merged_b.head()

,date,MUD PUMP B SLEECTED PRESSURE,MUD PUMP B SPEED.1,MUD PUMP B CYLINDER 1,MUD PUMP B CYLINDER 3,Mud Pump B Cylinder #1 Temperature,Mud Pump B Cylinder #2 Temperature,Mud Pump B Cylinder #3 Temperature,Mud Pump B Cylinder #4 Temperature,Mud Pump B,...,B_NORTH_OIL_SIDE.RESID_LIFE_%,B_NORTH_OIL_SIDE.RUNDAYS,B_SOUTH_OIL_SIDE.DAYS_LEFT,B_SOUTH_OIL_SIDE.EXPECTED_LIFE,B_SOUTH_OIL_SIDE.RESID_LIFE_%,B_SOUTH_OIL_SIDE.RUNDAYS,B_LAST_NORTH_OIL_SIDE_SM,B_LAST_SOUTH_OIL_SIDE_SM,B_NORTH_OIL_SIDE.RLIFE_STATUS,B_SOUTH_OIL_SIDE.RLIFE_STATUS
0,2024-09-03,6798.980143,95.987379,6907.886271,6847.133504,52.386997,47.326193,60.189480,45.148176,RUN,...,24.799681,274.522888,90.477127,365.0,24.799681,274.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE
1,2024-09-04,6605.291890,94.926737,6700.639771,6642.691895,52.447641,47.855699,60.016090,44.724799,RUN,...,24.525707,275.522888,89.477127,365.0,24.525707,275.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE
2,2024-09-05,6419.585948,91.162817,6520.643260,6461.790059,52.097924,46.509533,58.614579,43.891251,RUN,...,24.251732,276.522888,88.477127,365.0,24.251732,276.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE
3,2024-09-06,6805.291066,88.091589,6912.253713,6854.969137,53.058734,48.125668,59.280195,45.667776,RUN,...,23.977762,277.522888,87.477127,365.0,23.977762,277.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE
4,2024-09-07,6959.757548,93.627699,7067.157064,7008.055684,54.306744,48.706795,61.454095,46.979132,RUN,...,23.703787,278.522888,86.477127,365.0,23.703787,278.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE


In [89]:
vc = merged_a['Mud Pump A'].value_counts(normalize=True) * 100
vc = vc.round(2)
vc_df = vc.rename_axis('Mud Pump A').reset_index(name='percent')
vc_df

,Mud Pump A,percent
0,RUN,80.11
1,STOP,19.89


In [90]:
merged_b.head()

,date,MUD PUMP B SLEECTED PRESSURE,MUD PUMP B SPEED.1,MUD PUMP B CYLINDER 1,MUD PUMP B CYLINDER 3,Mud Pump B Cylinder #1 Temperature,Mud Pump B Cylinder #2 Temperature,Mud Pump B Cylinder #3 Temperature,Mud Pump B Cylinder #4 Temperature,Mud Pump B,...,B_NORTH_OIL_SIDE.RESID_LIFE_%,B_NORTH_OIL_SIDE.RUNDAYS,B_SOUTH_OIL_SIDE.DAYS_LEFT,B_SOUTH_OIL_SIDE.EXPECTED_LIFE,B_SOUTH_OIL_SIDE.RESID_LIFE_%,B_SOUTH_OIL_SIDE.RUNDAYS,B_LAST_NORTH_OIL_SIDE_SM,B_LAST_SOUTH_OIL_SIDE_SM,B_NORTH_OIL_SIDE.RLIFE_STATUS,B_SOUTH_OIL_SIDE.RLIFE_STATUS
0,2024-09-03,6798.980143,95.987379,6907.886271,6847.133504,52.386997,47.326193,60.189480,45.148176,RUN,...,24.799681,274.522888,90.477127,365.0,24.799681,274.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE
1,2024-09-04,6605.291890,94.926737,6700.639771,6642.691895,52.447641,47.855699,60.016090,44.724799,RUN,...,24.525707,275.522888,89.477127,365.0,24.525707,275.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE
2,2024-09-05,6419.585948,91.162817,6520.643260,6461.790059,52.097924,46.509533,58.614579,43.891251,RUN,...,24.251732,276.522888,88.477127,365.0,24.251732,276.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE
3,2024-09-06,6805.291066,88.091589,6912.253713,6854.969137,53.058734,48.125668,59.280195,45.667776,RUN,...,23.977762,277.522888,87.477127,365.0,23.977762,277.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE
4,2024-09-07,6959.757548,93.627699,7067.157064,7008.055684,54.306744,48.706795,61.454095,46.979132,RUN,...,23.703787,278.522888,86.477127,365.0,23.703787,278.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE


In [55]:
days_left_cols = [c for c in merged_a.columns if "days_left" in c.lower()]
merged_a["min_days_left"] = merged_a[days_left_cols].min(axis=1)
merged_a["fail_within_7"] = (merged_a["min_days_left"] <= 7).astype(int)

In [56]:
merged_b.columns.to_list()

['date',
 'MUD PUMP B SLEECTED PRESSURE',
 'MUD PUMP B SPEED.1',
 'MUD PUMP B CYLINDER 1',
 'MUD PUMP B CYLINDER 3',
 'Mud Pump B Cylinder #1 Temperature',
 'Mud Pump B Cylinder #2 Temperature',
 'Mud Pump B Cylinder #3 Temperature',
 'Mud Pump B Cylinder #4 Temperature',
 'Mud Pump B',
 'MUD PUMP B SPEED',
 'Mud Pump B Current',
 'MUD PUMP B SPEED.2',
 'B_NORTH_OIL_SIDE.DAYS_LEFT',
 'B_NORTH_OIL_SIDE.EXPECTED_LIFE',
 'B_NORTH_OIL_SIDE.RESID_LIFE_%',
 'B_NORTH_OIL_SIDE.RUNDAYS',
 'B_SOUTH_OIL_SIDE.DAYS_LEFT',
 'B_SOUTH_OIL_SIDE.EXPECTED_LIFE',
 'B_SOUTH_OIL_SIDE.RESID_LIFE_%',
 'B_SOUTH_OIL_SIDE.RUNDAYS',
 'B_LAST_NORTH_OIL_SIDE_SM',
 'B_LAST_SOUTH_OIL_SIDE_SM',
 'B_NORTH_OIL_SIDE.RLIFE_STATUS',
 'B_SOUTH_OIL_SIDE.RLIFE_STATUS']

In [57]:
print(merged_b.shape)


(367, 25)


In [58]:
print(merged_a['fail_within_7'].value_counts())


fail_within_7
1    367
Name: count, dtype: int64


In [59]:

def compute_pump_b_labels(df_b: pd.DataFrame) -> pd.DataFrame:
    """
    Given a DataFrame for Pump B data, compute:
     - B_NORTH_CALC_DAYS_LEFT, B_SOUTH_CALC_DAYS_LEFT
     - B_min_days_left (min across north/south, ignoring NaNs)
     - B_fail_within_7 (1 if min_days_left <= 7 else 0)

    Returns a copy of the dataframe with new columns added.
    """

    north_exp = "B_NORTH_OIL_SIDE.EXPECTED_LIFE"
    north_run = "B_NORTH_OIL_SIDE.RUNDAYS"
    south_exp = "B_SOUTH_OIL_SIDE.EXPECTED_LIFE"
    south_run = "B_SOUTH_OIL_SIDE.RUNDAYS"

    # Safety: check columns exist
    missing = [c for c in (north_exp, north_run, south_exp, south_run) if c not in df_b.columns]
    if missing:
        raise KeyError(f"Missing expected Pump B columns: {missing}")

    # Convert to numeric (coerce non-numeric -> NaN)
    for col in (north_exp, north_run, south_exp, south_run):
        df_b[col] = pd.to_numeric(df_b[col], errors="coerce")

    # 1) Compute DAYS_LEFT for north and south
    df_b["B_NORTH_CALC_DAYS_LEFT"] = df_b[north_exp] - df_b[north_run]
    df_b["B_SOUTH_CALC_DAYS_LEFT"] = df_b[south_exp] - df_b[south_run]

    # 2) Compute minimum days left across components (ignores NaNs if one is missing)
    df_b["B_min_days_left"] = df_b[["B_NORTH_CALC_DAYS_LEFT", "B_SOUTH_CALC_DAYS_LEFT"]].min(axis=1, skipna=True)

    # 3) Create binary label: fail within 7 days (includes negative = overdue)
    df_b["B_fail_within_7"] = (df_b["B_min_days_left"] <= 7).astype(int)

    return df_b


In [60]:
compute_pump_b_labels(merged_b)

,date,MUD PUMP B SLEECTED PRESSURE,MUD PUMP B SPEED.1,MUD PUMP B CYLINDER 1,MUD PUMP B CYLINDER 3,Mud Pump B Cylinder #1 Temperature,Mud Pump B Cylinder #2 Temperature,Mud Pump B Cylinder #3 Temperature,Mud Pump B Cylinder #4 Temperature,Mud Pump B,...,B_SOUTH_OIL_SIDE.RESID_LIFE_%,B_SOUTH_OIL_SIDE.RUNDAYS,B_LAST_NORTH_OIL_SIDE_SM,B_LAST_SOUTH_OIL_SIDE_SM,B_NORTH_OIL_SIDE.RLIFE_STATUS,B_SOUTH_OIL_SIDE.RLIFE_STATUS,B_NORTH_CALC_DAYS_LEFT,B_SOUTH_CALC_DAYS_LEFT,B_min_days_left,B_fail_within_7
0,2024-09-03,6798.980143,95.987379,6907.886271,6847.133504,52.386997,47.326193,60.189480,45.148176,RUN,...,24.799681,274.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE,90.477112,90.477112,90.477112,0
1,2024-09-04,6605.291890,94.926737,6700.639771,6642.691895,52.447641,47.855699,60.016090,44.724799,RUN,...,24.525707,275.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE,89.477112,89.477112,89.477112,0
2,2024-09-05,6419.585948,91.162817,6520.643260,6461.790059,52.097924,46.509533,58.614579,43.891251,RUN,...,24.251732,276.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE,88.477112,88.477112,88.477112,0
3,2024-09-06,6805.291066,88.091589,6912.253713,6854.969137,53.058734,48.125668,59.280195,45.667776,RUN,...,23.977762,277.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE,87.477112,87.477112,87.477112,0
4,2024-09-07,6959.757548,93.627699,7067.157064,7008.055684,54.306744,48.706795,61.454095,46.979132,RUN,...,23.703787,278.522888,2023-07-12,2023-07-12,ON_LINE,ON_LINE,86.477112,86.477112,86.477112,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,2025-08-31,7223.414856,-5.000000,2.691820,-33.728752,15.411098,15.085702,15.010597,14.703357,STOP,...,55.195938,163.534821,2024-12-27,2024-12-27,ON_LINE,ON_LINE,201.465179,201.465179,201.465179,0
363,2025-09-01,7115.739054,-5.000000,2.555222,-34.315102,15.588098,15.257933,15.728601,15.276235,STOP,...,55.195938,163.534821,2024-12-27,2024-12-27,ON_LINE,ON_LINE,201.465179,201.465179,201.465179,0
364,2025-09-02,6591.118713,-5.000000,2.378113,-33.916207,15.469199,14.926395,15.312697,14.779955,STOP,...,55.195938,163.534821,2024-12-27,2024-12-27,ON_LINE,ON_LINE,201.465179,201.465179,201.465179,0
365,2025-09-03,6439.783223,40.575959,3980.273093,3930.559525,35.222132,36.538742,35.024102,34.097240,STOP,...,55.195938,163.534821,2024-12-27,2024-12-27,ON_LINE,ON_LINE,201.465179,201.465179,201.465179,0


In [61]:
merged_b['B_fail_within_7'].value_counts()

B_fail_within_7
0    350
1     17
Name: count, dtype: int64

In [62]:

merged_a.columns = [
    "date",
    "react_55_mud_flow_to_mud_pump_a",
    "a_suction_pressure",
    "a_speed_raw",
    "a_cylinder_2",
    "a_cylinder_3",
    "a_cylinder_1_temp",
    "a_cylinder_2_temp",
    "a_cylinder_3_temp",
    "a_cylinder_4_temp",
    "a_pressure",
    "a_speed",
    "a_current",
    "a_speed_alt",
    "a_north_mini_oil_side_days_left",
    "a_north_mini_oil_side_expected_life",
    "a_north_mini_oil_side_resid_life_pct",
    "a_north_oil_side_days_left",
    "a_north_oil_side_expected_life",
    "a_north_oil_side_resid_life_pct",
    "a_north_oil_side_rundays",
    "a_south_mini_oil_side_days_left",
    "a_south_mini_oil_side_expected_life",
    "a_south_mini_oil_side_resid_life_pct",
    "a_south_oil_side_days_left",
    "a_south_oil_side_expected_life",
    "a_south_oil_side_resid_life_pct",
    "a_south_oil_side_rundays",
    "a_mud_side_days_left",
    "a_mud_side_expected_life",
    "a_mud_side_resid_life_pct",
    "a_north_last_mini_oil_side_sm",
    "a_north_last_oil_side_sm",
    "a_north_mini_oil_side_rlife_status",
    "a_north_oil_side_rlife_status",
    "a_south_last_mini_oil_side_sm",
    "a_south_last_oil_side_sm",
    "a_south_mini_oil_side_rlife_status",
    "a_south_oil_side_rlife_status",
    "a_last_mud_side_sm",
    "a_mud_side_rlife_status",
    "a_min_days_left",
    "a_fail_within_7",
]

merged_b.columns = [
    "date",
    "b_selected_pressure",
    "b_speed_raw",
    "b_cylinder_1",
    "b_cylinder_3",
    "b_cylinder_1_temp",
    "b_cylinder_2_temp",
    "b_cylinder_3_temp",
    "b_cylinder_4_temp",
    "b_pressure",
    "b_speed",
    "b_current",
    "b_speed_alt",
    "b_north_oil_side_days_left",
    "b_north_oil_side_expected_life",
    "b_north_oil_side_resid_life_pct",
    "b_north_oil_side_rundays",
    "b_south_oil_side_days_left",
    "b_south_oil_side_expected_life",
    "b_south_oil_side_resid_life_pct",
    "b_south_oil_side_rundays",
    "b_last_north_oil_side_sm",
    "b_last_south_oil_side_sm",
    "b_north_oil_side_rlife_status",
    "b_south_oil_side_rlife_status",
    "b_north_calc_days_left",
    "b_south_calc_days_left",
    "b_min_days_left",
    "b_fail_within_7",
]





In [63]:
final_merged = merged_a.merge(merged_b, on='date', how='inner')

new_merged = final_merged[
    [c for c in final_merged.columns 
     if "expected_life" not in c.lower() and "rlife" not in c.lower()]
]
new_merged.head()

,date,react_55_mud_flow_to_mud_pump_a,a_suction_pressure,a_speed_raw,a_cylinder_2,a_cylinder_3,a_cylinder_1_temp,a_cylinder_2_temp,a_cylinder_3_temp,a_cylinder_4_temp,...,b_north_oil_side_rundays,b_south_oil_side_days_left,b_south_oil_side_resid_life_pct,b_south_oil_side_rundays,b_last_north_oil_side_sm,b_last_south_oil_side_sm,b_north_calc_days_left,b_south_calc_days_left,b_min_days_left,b_fail_within_7
0,2024-09-03,124.984502,397.001902,95.775800,6775.664246,6801.475850,62.122698,52.474178,61.062826,57.914323,...,274.522888,90.477127,24.799681,274.522888,2023-07-12,2023-07-12,90.477112,90.477112,90.477112,0
1,2024-09-04,104.673728,355.022669,94.778817,6579.124349,6603.233887,62.024733,52.407360,60.802001,56.761674,...,275.522888,89.477127,24.525707,275.522888,2023-07-12,2023-07-12,89.477112,89.477112,89.477112,0
2,2024-09-05,96.006373,358.586122,90.898222,6390.380920,6415.289510,60.754139,50.022266,59.948708,54.314469,...,276.522888,88.477127,24.251732,276.522888,2023-07-12,2023-07-12,88.477112,88.477112,88.477112,0
3,2024-09-06,88.442687,372.479466,87.573452,6779.555349,6803.085571,61.347060,51.532399,60.425867,55.867826,...,277.522888,87.477127,23.977762,277.522888,2023-07-12,2023-07-12,87.477112,87.477112,87.477112,0
4,2024-09-07,99.902062,373.857407,93.896962,6934.170593,6958.844666,63.144880,54.056685,62.013078,58.409987,...,278.522888,86.477127,23.703787,278.522888,2023-07-12,2023-07-12,86.477112,86.477112,86.477112,0


In [64]:
# indentified and removing leakage columns
leak_keywords = [
    "days_left",
    "expected_life",
    "resid_life",
    "calc_days",
    "min_days_left",
    "rlife_status",
    "last_",
    "north_oil_side_rundays",
    "north_oil_side_rundays",
    "south_oil_side_rundays"      
]

new_merged_no_leak = new_merged[[c for c in new_merged.columns if not any(k in c.lower() for k in leak_keywords)]]



In [65]:
new_merged_no_leak.shape

(367, 28)

In [66]:
new_merged_no_leak.columns.to_list()

['date',
 'react_55_mud_flow_to_mud_pump_a',
 'a_suction_pressure',
 'a_speed_raw',
 'a_cylinder_2',
 'a_cylinder_3',
 'a_cylinder_1_temp',
 'a_cylinder_2_temp',
 'a_cylinder_3_temp',
 'a_cylinder_4_temp',
 'a_pressure',
 'a_speed',
 'a_current',
 'a_speed_alt',
 'a_fail_within_7',
 'b_selected_pressure',
 'b_speed_raw',
 'b_cylinder_1',
 'b_cylinder_3',
 'b_cylinder_1_temp',
 'b_cylinder_2_temp',
 'b_cylinder_3_temp',
 'b_cylinder_4_temp',
 'b_pressure',
 'b_speed',
 'b_current',
 'b_speed_alt',
 'b_fail_within_7']

In [67]:
def combine_pumps(df):
    """
    Combine Pump A and Pump B data into a single long-format dataframe.
    
    - Detects a_ and b_ columns automatically
    - Removes prefixes
    - Adds pump_id (A or B)
    - Returns clean combined dataframe
    """
    import pandas as pd
    
    # ensure date is datetime
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])
    
    # detect A and B columns
    cols_a = [c for c in df.columns if c.startswith("a_")]
    cols_b = [c for c in df.columns if c.startswith("b_")]

    # build A dataframe
    df_a = df[["date"] + cols_a].copy()
    df_a.columns = ["date"] + [c.replace("a_", "") for c in cols_a]
    df_a["pump_id"] = "A"

    # build B dataframe
    df_b = df[["date"] + cols_b].copy()
    df_b.columns = ["date"] + [c.replace("b_", "") for c in cols_b]
    df_b["pump_id"] = "B"
    
    # concatenate
    df_all = pd.concat([df_a, df_b], ignore_index=True)
    
    # sort properly
    df_all = df_all.sort_values(["date", "pump_id"]).reset_index(drop=True)
    
    return df_all


In [68]:
df_all = combine_pumps(new_merged_no_leak)

df_all.head()

,date,suction_pressure,speed_raw,cylinder_2,cylinder_3,cylinder_1_temp,cylinder_2_temp,cylinder_3_temp,cylinder_4_temp,pressure,speed,current,speed_alt,fail_within_7,pump_id,selected_pressure,cylinder_1
0,2024-09-03,397.001902,95.775800,6775.664246,6801.475850,62.122698,52.474178,61.062826,57.914323,RUN,42.003342,262.888275,41.446602,1,A,NaN,NaN
1,2024-09-03,NaN,95.987379,NaN,6847.133504,52.386997,47.326193,60.189480,45.148176,RUN,41.921871,32.062214,41.921871,0,B,6798.980143,6907.886271
2,2024-09-04,355.022669,94.778817,6579.124349,6603.233887,62.024733,52.407360,60.802001,56.761674,RUN,42.026459,267.0112,40.453312,1,A,NaN,NaN
3,2024-09-04,NaN,94.926737,NaN,6642.691895,52.447641,47.855699,60.016090,44.724799,RUN,42.114395,34.593483,42.114395,0,B,6605.291890,6700.639771
4,2024-09-05,358.586122,90.898222,6390.380920,6415.289510,60.754139,50.022266,59.948708,54.314469,RUN,42.043556,270.136536,38.511074,1,A,NaN,NaN


In [69]:
# want to view the numerical and categorical
num_cols = df_all.select_dtypes(include=['int64','float']).columns
display(df_all[num_cols].describe().T)

# categorical
cat_cols = df_all.select_dtypes(include=['object','category']).columns
for col in cat_cols:
    print(f"\nValue counts for {col}:")
    counts = df_all[col].value_counts(dropna=False)
    percentages = counts / len(df_all) * 100
    display(pd.DataFrame({'count': counts, 'percentage': percentages.round(2)}))

,count,mean,std,min,25%,50%,75%,max
suction_pressure,367.0,339.981707,165.660577,0.559934,313.077790,391.693111,448.119936,585.573259
speed_raw,734.0,76.547506,36.222621,-5.000000,82.799790,93.287062,98.502167,100.000000
cylinder_2,367.0,4903.341473,2413.085321,-48.170810,4549.695938,6056.744832,6555.546285,7690.230469
cylinder_3,734.0,4607.150683,2626.167678,-34.315102,2870.840290,5950.985158,6567.095464,7758.225586
cylinder_1_temp,734.0,44.749821,16.888797,5.668491,34.820565,51.611889,57.566660,63.928203
cylinder_2_temp,734.0,41.483170,15.468755,4.949737,33.455018,46.900239,51.861873,61.433031
cylinder_3_temp,734.0,42.661930,15.864526,5.752025,33.212795,49.540546,53.436163,62.013078
cylinder_4_temp,734.0,40.492062,15.225710,5.666300,31.245432,46.652472,52.460348,58.409987
fail_within_7,734.0,0.523161,0.499804,0.000000,0.000000,1.000000,1.000000,1.000000
selected_pressure,367.0,4930.120425,2411.995860,-21.422602,4576.899244,6083.015096,6588.321869,7732.564941



Value counts for pressure:


,count,percentage
pressure,,
RUN,547,74.52
STOP,187,25.48



Value counts for speed:


,count,percentage
speed,,
Bad,102,13.90
-0.040396,15,2.04
-0.043546,6,0.82
-0.037245,6,0.82
-0.040613,3,0.41
...,...,...
42.052498,1,0.14
-0.042297,1,0.14
42.014553,1,0.14


...

,count,percentage
current,,
Bad,102,13.90
-0.242363,3,0.41
0.218932,3,0.41
0.271893,3,0.41
-0.161499,3,0.41
...,...,...
0.236586,1,0.14
260.694244,1,0.14
0.232025,1,0.14



Value counts for speed_alt:


,count,percentage
speed_alt,,
Bad,61,8.31
Bad Total,52,7.08
-0.040396,15,2.04
-0.043546,6,0.82
-0.037245,6,0.82
...,...,...
41.935204,1,0.14
41.671005,1,0.14
-0.042297,1,0.14



Value counts for pump_id:


,count,percentage
pump_id,,
A,367,50.0
B,367,50.0


In practical industrial settings, pump current and speeds should never be negative.
During normal operation, electrical systems consume power — they do not generate or push power back into the grid. Therefore, a negative current or speed value is physically impossible for Mud Pump A, Mud Pump B, or any similar industrial pump.

these may be due to either Sensor malfunction or miscalibration,Data logging or transmission errors,Incorrect preprocessing in the source system


In [70]:
# Identify and fix all speed and current columns
speed_cols = [c for c in df_all.columns if "speed" in c.lower()]
current_cols = [c for c in df_all.columns if "current" in c.lower()]

df_all[speed_cols] = df_all[speed_cols].apply(pd.to_numeric, errors='coerce')
df_all[current_cols] = df_all[current_cols].apply(pd.to_numeric, errors='coerce')

df_all[speed_cols] = df_all[speed_cols].clip(lower=0)
df_all[current_cols] = df_all[current_cols].clip(lower=0)


df_all[speed_cols] = df_all[speed_cols].interpolate()
df_all[current_cols] = df_all[current_cols].interpolate()


In [71]:
df_all = df_all.sort_values("date").reset_index(drop=True)

target = "fail_within_7"
X = df_all.drop(columns=[target, "date","pressure","pump_id","cylinder_2","suction_pressure","current"])
y = df_all[target]

# 4. Identify numeric and categorical columns CORRECTLY
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns  


In [72]:
#feature engineering pipeline

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scale", StandardScaler()),
    ("drop_corr", DropCorrelatedFeatures(threshold=0.7))
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])



feature_eng_pipeline = ColumnTransformer(
    [
        ("numeric", numeric_transformer, num_cols),
        ("categorical", categorical_transformer, cat_cols)
    ],
    remainder="drop",
    verbose_feature_names_out=False
)


In [73]:

model_xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    tree_method="hist",
    random_state=42
)


In [74]:
#final pipeline
model_xgb = Pipeline([
    ("features", feature_eng_pipeline),
    ("xgb", model_xgb)
])


In [91]:
# Time Series Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)
fold = 1

for train_idx, test_idx in tscv.split(X):
    print(f"\n====== FOLD {fold} ======")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model_xgb.fit(X_train, y_train)
    preds = model_xgb.predict(X_test)

    print(classification_report(y_test, preds))
    fold += 1


====== FOLD 1 ======
              precision    recall  f1-score   support

           0       0.77      1.00      0.87        47
           1       1.00      0.81      0.90        75

    accuracy                           0.89       122
   macro avg       0.89      0.91      0.88       122
weighted avg       0.91      0.89      0.89       122


====== FOLD 2 ======
              precision    recall  f1-score   support

           0       0.95      0.98      0.97        58
           1       0.98      0.95      0.97        64

    accuracy                           0.97       122
   macro avg       0.97      0.97      0.97       122
weighted avg       0.97      0.97      0.97       122


====== FOLD 3 ======
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        61
           1       0.98      1.00      0.99        61

    accuracy                           0.99       122
   macro avg       0.99      0.99      0.99       122
weighted

In [76]:
df_all.groupby("fail_within_7")[num_cols].mean()


,speed_raw,cylinder_3,cylinder_1_temp,cylinder_2_temp,cylinder_3_temp,cylinder_4_temp,speed,speed_alt,selected_pressure,cylinder_1
fail_within_7,,,,,,,,,,
0,63.024615,4239.881833,39.698254,36.620501,38.565296,35.828551,33.862706,33.797080,4846.948783,4280.201892
1,90.202810,4941.900938,49.354113,45.915290,46.395841,44.742658,36.122569,35.948119,6642.477758,5184.933797


In [77]:
for col in X.columns:
    print(col, df_all[col].corr(df_all["fail_within_7"]))


speed_raw 0.3918904904023376
cylinder_3 0.13360604144419339
cylinder_1_temp 0.28575367104231103
cylinder_2_temp 0.3003196685470531
cylinder_3_temp 0.2466973341672896
cylinder_4_temp 0.2926172408555459
speed 0.08232832686646319
speed_alt 0.08332441895304467
selected_pressure 0.15667538405295423
cylinder_1 0.06780709725532127
